In [1]:
!pip -q install ../input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl

In [2]:
#Import packages
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from itertools import compress
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  
from sklearn.metrics import precision_score                         
from sklearn.metrics import recall_score
import lightgbm
import optuna
from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.matlib

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

from scipy import stats
from scipy.stats import norm
from joblib import Parallel, delayed

import shutil
import glob

from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold

# from pytorch_tabnet.metrics import Metric
# from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts


In [3]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import roc_auc_score

from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

import optuna

from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Concatenate, LSTM, GRU
from tensorflow.keras.layers import Bidirectional, Multiply


np.random.seed(2022)
tf.random.set_seed(2022)

In [4]:
train_data = pd.read_parquet('../input/amex-parquet/train_data.parquet')

In [5]:
train=train_data.groupby('customer_ID').tail(1)
del train_data
train=train.set_index(['customer_ID'])
train.drop(['S_2'],axis=1,inplace=True)

In [6]:
train=train.dropna(axis=1, thresh=int(0.80*len(train)))

In [7]:
train_D63 = pd.get_dummies(train[['D_63']])
train = pd.concat([train, train_D63], axis=1)
del train_D63
train = train.drop(['D_63'], axis=1)

train_D64 = pd.get_dummies(train[['D_64']])

train = pd.concat([train, train_D64], axis=1)
del train_D64
train = train.drop(['D_64'], axis=1)

In [8]:
train_without_target=train.drop(['target'],axis=1)
cor_matrix = train_without_target.corr().abs()
upper_tri = cor_matrix.where((np.triu(np.ones(cor_matrix.shape), k=1) + np.tril(np.ones(cor_matrix.shape), k=-1)).astype(bool))

to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.90)]
train_drop_highcorr=train_without_target.drop(to_drop,axis=1)
train_drop_highcorr.shape

In [9]:
del train_without_target, cor_matrix, upper_tri

In [10]:
def fs_variance(df, threshold:float=0.06):

    
    vt = VarianceThreshold(threshold = threshold)
    _ = vt.fit(df)
    
    feat_select = list(compress( list(df.columns), vt.get_support()))
    return feat_select
columns_to_keep=fs_variance(train_drop_highcorr)

train_final=train[columns_to_keep]

In [11]:

train_final1=train_final.join(train['target'])
x_train=train_final1.drop(['target'],axis=1)
y_train=train_final1['target']

In [ ]:
columns_to_keep

In [12]:
del train_final, train_final1,train

In [13]:
x_train.isnull().sum()[x_train.isnull().sum() > 0]

In [14]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

def lgb_amex_metric(y_pred, y_true):
#     y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

In [15]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(strategy='mean')
imp_mean.fit(x_train)

In [16]:
imp_mean.n_features_in_

In [17]:
x_train = imp_mean.transform(x_train)

In [18]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=0)
del x_train, y_train

In [19]:
def get_model():


    inp = tf.keras.layers.Input(shape=(76), name='inp')

    x=tf.keras.layers.Dense(512, 
                            kernel_regularizer=tf.keras.regularizers.l2(l=0.01),
                            activation='relu')(inp)
    x=tf.keras.layers.Dropout(0.2)(x)

    x=tf.keras.layers.Dense(256, 
                            kernel_regularizer=tf.keras.regularizers.l2(l=0.01),
                            activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)

    x=tf.keras.layers.Dense(128, 
                            kernel_regularizer=tf.keras.regularizers.l2(l=0.01),
                            activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)

    x=tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(l=0.01),
                            activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)

    y=tf.keras.layers.Dense(100, 
                            kernel_regularizer=tf.keras.regularizers.l2(l=0.01),
                            activation='relu')(x)

    output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(y)

    model = tf.keras.models.Model(inputs=inp, outputs=[output])


    return model

model = get_model()

plot_model(
    model, 
    to_file='Super_Model.png', 
    show_shapes=False,
    show_layer_names=True
)

In [21]:
AUTO = tf.data.experimental.AUTOTUNE
batch_size=32

from time import time
index=time()

tf.keras.backend.clear_session()
models = []
historys = []

checkpoint_path = "model_%d.h5"%(index)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    
    checkpoint_path, 
    verbose=1,
    save_best_only=True
)
early_stop = tf.keras.callbacks.EarlyStopping(
    min_delta=1e-4, 
    patience=10,
    verbose=1,

)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    factor=0.3,
    patience=2, 
    min_lr=1e-7,
     verbose=1,
        monitor='val_loss',
)
callbacks = [early_stop, checkpoint, reduce_lr]

optimizer = tf.keras.optimizers.SGD(1e-3)

train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(69).batch(batch_size).cache().prefetch(AUTO)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(batch_size).cache().prefetch(AUTO)
# del X_train, X_val, Y_train, Y_val

In [22]:

model = get_model()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True) , optimizer=optimizer, metrics=[tf.keras.metrics.AUC()])

history = model.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=callbacks)

for metrics in [("binary_crossentropy", "val_binary_crossentropy"), ("auc", "val_auc"), ["lr"]]:
    pd.DataFrame(history.history, columns=metrics).plot()
    plt.show()
    
model.load_weights(checkpoint_path)
historys.append(history)
models.append(model)

In [63]:
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [31]:
len(columns_to_keep) == len(not_dummy) + len(dummy)

In [43]:
not_dummy=[]
dummy=[]
for i in columns_to_keep:
    if 'D_63' in i or 'D_64' in i:
        dummy.append(i)
    else:
        not_dummy.append(i)

In [44]:
not_dummy.extend(["customer_ID","D_63","D_64"])

In [45]:
test_data = pd.read_parquet('../input/amex-parquet/test_data.parquet',columns=not_dummy)

In [46]:
test=test_data.groupby('customer_ID').tail(1)
del test_data
test=test.set_index(['customer_ID'])
# test.drop(['S_2'],axis=1,inplace=True)

test_D63 = pd.get_dummies(test[['D_63']])
test = pd.concat([test, test_D63], axis=1)
del test_D63
test = test.drop(['D_63'], axis=1)

test_D64 = pd.get_dummies(test[['D_64']])
test = pd.concat([test, test_D64], axis=1)
del test_D64
test = test.drop(['D_64'], axis=1)

In [50]:
preds = model.predict(imp_mean.transform(test[columns_to_keep]))
preds

In [51]:
df_sub = pd.read_csv('../input/amex-default-prediction/sample_submission.csv')
df_sub.prediction = preds

# df_sub = df_sub.set_index('customer_ID')
df_sub.to_csv('submission_nn.csv')